# Setup

In [1]:
%%capture
%pip install poetry
%pip install git+https://github.com/oughtinc/ergo.git@993c959d4b5dc5b398f02b842407814b79619a42
%pip install xlrd

In [1]:
%load_ext google.colab.data_table

In [1]:
%%capture
import ergo
import numpy as np
import pandas as pd
import ssl
import math
import datetime
import warnings
import functools
from datetime import timedelta, date
from ergo.contrib.el_paso import *

In [1]:
warnings.filterwarnings(module="plotnine", action="ignore")
warnings.filterwarnings(module="jax", action="ignore")
ssl._create_default_https_context = ssl._create_unverified_context

In [1]:
metaculus = ergo.Metaculus(
    username="oughtpublic", 
    password="123456",
    api_domain = "pandemic"
)

# Ergo extensions



We'll define some helper functions that might get moved into Ergo in the
future.



In [1]:
START_DATE = date(2020, 4, 1)


def daterange(start_date, end_date):
    for n in range(int((end_date - start_date).days)):
        yield start_date + timedelta(n)

        
# Rejection sampling

def rejection_sample(fn, condition):
    """
    Sample from fn until we get a value that satisfies
    condition, then return it.
    """
    while True:        
        value = fn()
        if condition(value):
            return value


# Memoization

memoized_functions = []

def mem(func):
    func = functools.lru_cache(None)(func)
    memoized_functions.append(func)
    return func

def clear_mem():
    for func in memoized_functions:
        func.cache_clear()

        
# Associate models with questions

# We'll add a sampler here for each question we predict on. 
# Each sampler is a function that returns a single sample
# from our model predicting on that question.
samplers = {}

def question(question_id, community_weight=0, community_fn=None):
    q = metaculus.get_question(question_id)

    def decorator(func):
        tag = func.__name__

        @functools.wraps(func)
        @mem
        def sampler():
            if ergo.flip(community_weight):
                if community_fn:
                    value = community_fn()
                else:
                    value = q.sample_community()
            else:
                value = func()
            if isinstance(value, date):
                # FIXME: Ergo needs to handle dates
                ergo.tag(int((value - START_DATE).days), tag)
            else:
                ergo.tag(value, tag)
            return value
        sampler.question = q
        samplers[q.id] = sampler
        return sampler
    return decorator

def summarize_question_samples(samples):
    sampler_tags = [sampler.__name__ for sampler in samplers.values()]
    tags_to_show = [tag for tag in sampler_tags if tag in samples.columns]
    samples_to_show = samples[tags_to_show]
    summary = samples_to_show.describe().transpose().round(2)
    display(summary)


def sample_from_ensemble(models, weights):
    """
    Sample models in proportion to weights
    """
    return lambda : ergo.random_choice(models, weights)


def plot_question(sampler, num_samples=200, bw=None):
  def model():
      clear_mem()
      sampler()

  samples = ergo.run(model, num_samples=num_samples)

  summarize_question_samples(samples)

  q = sampler.question

  q_samples = samples[sampler.__name__]

  if q.id == 4128: # Date question: Need to convert back to date from days (https://github.com/oughtinc/ergo/issues/144)
      q_samples = np.array([START_DATE + timedelta(s) for s in q_samples])

  if bw is not None:
      q.show_prediction(samples=q_samples, show_community=True, percent_kept=0.9, bw=bw)
  else:
      q.show_prediction(samples=q_samples, show_community=True, percent_kept=0.9)

# External data (cases, estimates, models)



## Texas government cases data



In [1]:
el_paso_cases = texas_data.get_el_paso_data()

el_paso_cases

## @onlyasith's cases model



Pulled from [here](https://docs.google.com/spreadsheets/d/1L6pzFAEJ6MfnUwt-ea6tetKyvdi0YubnK_70SGm436c/edit#gid=1807978187)



In [1]:
projected_cases = onlyasith.get_onlyasith_results()

projected_cases

## Shaman et al. model of confirmed cases



[research article](https://www.medrxiv.org/content/10.1101/2020.03.21.20040303v2)

Pulled from [here](https://github.com/shaman-lab/COVID-19Projection)



In [1]:
scenarios = ["Projection_nointerv", "Projection_60contact", "Projection_70contact", "Projection_80contact"]
cu_model_data = {}
for scenario in scenarios:
  df = pd.read_csv(f"https://raw.githubusercontent.com/shaman-lab/COVID-19Projection/master/Projection_April26/{scenario}.csv", parse_dates=["Date"])
  df = df[df.county == "El Paso County TX"]
  df["Date"] = df["Date"].apply(lambda x: x.date())
  df.set_index("Date", inplace = True)

  cu_model_data[scenario] = df


@mem
def cu_model_scenario():
  """Which of the model scenarios are we in?"""
  return ergo.random_choice([s for s in cu_model_data.keys()])

@mem
def cu_model_quantile():
  """Where in the distribution of model outputs are we for this model run?

  Want to be consistent across time, so we sample it once per model run"""
  return ergo.uniform()

#df = pd.read_csv(f"https://raw.githubusercontent.com/shaman-lab/COVID-19Projection/master/Projection_April26/Projection_nointerv.csv", parse_dates=["Date"])
#df = pd.read_csv(f"https://raw.githubusercontent.com/shaman-lab/COVID-19Projection/master/Projection_April26/{scenario}.csv", index_col="Date", parse_dates=True)

## @KrisMoore's compiled data



Pulled from [here](https://docs.google.com/spreadsheets/d/1eGF9xYmDmvAkr-dCmd-N4efHzPyYEfVl0YmL9zBvH9Q/edit#gid=1694267458)



In [1]:
compiled_data = krismoore.get_krismoore_data()

compiled_data

## @brachbach model (cases -> hospitalized)



In [1]:
get_daily_hospital_confirmed = brachbach.get_daily_hospital_confirmed

# Model components



Many of the Metaculus questions are asking to quantify the result of complex causal processes. To answer 'How many total patients are in the ICU on [date]?' requires a specification of which factors lead to an ICO patient&#x2014;from the societal processes that are adopted -> exposure risk -> disease development trajectory -> # icu patients. We adopt the approach of trying to explicitly specify the causal process underlying each question. In this section we model some of the relevant variables we will make use of in the next section's models. We will take an ensemble of models approach. We will therefore sometimes specify multiple models for each variable we are interested in. These models can be mixed together to (hopefully) result in more robust predictions. Ultimately the mixture parameter would be a random variable conditioned on the model's success. At the moment it is a parameter for the modeler to explicitly tune.



## Daily Infections



Shaman Model



In [1]:
def daily_infections_cu_model(date: date) -> int:
    """
    Predict the number of reported (new) Covid-19 infections on [date]
    using the Columbia model
    """
    scenario = cu_model_scenario()
    quantile = cu_model_quantile()

    # Extract quantiles of the model distribution
    xs = np.array([0.025, 0.25, 0.5, 0.75, 0.975])
    ys = np.array([
        cu_model_data[scenario][s][date] for s in ["report_2.5", "report_25", "report_50", "report_75", "report_97.5"]
    ])
    # Linearly interpolate
    return np.interp(quantile, xs, ys)

onlyasith model+



In [1]:
@mem
def daily_infections_base_model(date: date) -> int:
    """
    What is the number of reported (new) Covid-19 infections on [date]?
    """
    try:
        # Look up projections from @onlyasith's model
        cases = projected_cases.loc[date, "New cases"]
        if np.isnan(cases):
            raise KeyError

        # Add some (fairly arbitrary) uncertainty around this point estimate
        if cases == 0:
          return cases
      cases_estimate = ergo.lognormal_from_interval(cases * 0.8, cases * 1.2)
        return np.clip(cases_estimate, cases * 0.5, cases * 2)
        except KeyError:
            # We're beyond the time range for data and model

Sample from ensemble



In [1]:
@mem
def daily_infections_v1(date: date) -> int:
    """
    What is the number of reported (new) Covid-19 infections on [date]?
    """
    try:
      infections = el_paso_cases.loc[date, "New cases"]
      if np.isnan(infections):
        raise KeyError
      return infections
    except KeyError:
      model = sample_from_ensemble([daily_infections_cu_model, daily_infections_base_model], [.8, .2])
      infections = model(date)
        if np.isnan(infections):
          raise KeyError
        return int(round(infections))
      except KeyError:
        print("we shouldn't be cool with this")
        continue
    return 0 # should we be cool with this?

Sample from ensemble original



In [1]:
def predict_daily_infections(date: date) -> int:
    """
    Predict the number of reported (new) Covid-19 infections on [date].
    """

    @mem
    def get_infections_models_by_preference():
      """
      Set a primary and secondary model.
      We need the secondary model to fail over toin case the primary model is missing data.
      Memoize this preference ordering so that we use the same primary model
      throughout any given run.
      """
      if ergo.flip(0.7):
        return [predict_daily_infections_cu_model, lambda date: onlyasith_cases.loc[date, "New cases"]]

      return [lambda date: onlyasith_cases.loc[date, "New cases"], predict_daily_infections_cu_model]

    models_by_preference = get_infections_models_by_preference()

    for model in models_by_preference:
      try:
        infections = model(date)
        if np.isnan(infections):
          raise KeyError
        return int(round(infections))
      except KeyError:
        continue

    return 0

@mem
def daily_infections_v0(date: date) -> int:
    """
    What is the number of reported (new) Covid-19 infections on [date]?
    """
    try:
      new_cases = el_paso_cases.loc[date, "New cases"]
      if np.isnan(new_cases):
        raise KeyError
      return new_cases
    except KeyError:
      return predict_daily_infections(date)

In [1]:
@mem
def mean_infections(start_date: date, end_date: date):
    """
    What is the average number of reported new infections for this range of 
    dates? (Including start date, excluding end date)
    """
    days = daterange(start_date, end_date)
    return np.mean([daily_infections_v0(day) for day in days])

## Confirmed Hospital Patients



Model from Shaman et al.



In [1]:
@mem
def get_hospital_confirmed_from_daily_infected_model():
    
    # from https://penn-chime.phl.io/
    hospital_stay_days_point_estimate = 7

    hospital_stay_days_fuzzed = round(
        float(ergo.normal_from_interval(
            hospital_stay_days_point_estimate * 0.5,
            hospital_stay_days_point_estimate * 1.5
        ))
    )

    hospital_stay_days = max(1, hospital_stay_days_fuzzed)

    has_hospital_confirmed = compiled_data[compiled_data["In hospital confirmed"].notna()]

    data_dates = has_hospital_confirmed.index

    # for each date for which we have data for how many lab-confirmed COVID patients were in the hospital,
    # how many new confirmed cases were there over the past hospital_stay_days days?
    def get_recent_infected_data(date):
      return sum([daily_infections_v0(date - timedelta(n))
        for n in range(0, hospital_stay_days)])

    recent_infected_data = [[get_recent_infected_data(date)]
      for date in data_dates]

    reg = LinearRegression(fit_intercept=False).fit(
        recent_infected_data,
        has_hospital_confirmed["In hospital confirmed"])

    # TODO: consider adding uncertainty to the fit here

    # now that we've related current hospitalized cases and recent confirmed cases,
    # return a function that allows us to predict hospitalized cases given estimates
    # of future confirmed cases
    def hospital_confirmed_from_daily_infected_model(date: date):
      recent_infected = sum([daily_infections_v0(date - timedelta(n))
        for n in range(0, hospital_stay_days)])
      return int(round(reg.predict([[recent_infected]])[0]))
  
    return hospital_confirmed_from_daily_infected_model

@mem
def hospital_confirmed_for_date(date: date) -> int:
    """
    The total number of lab-confirmed COVID-19 
    patients in El Paso County in the hospital on this date
    """

    # We predict the number of lab-confirmed COVID patients
    # in the hospital on some date
    # as some multiple of the number of new confirmed COVID cases
    # over the past 7 or so days
    # (since someone who gets admitted to the hospital for COVID
    # will probably stay there for 7 or so days)

    hospital_confirmed_from_daily_infected_model = get_hospital_confirmed_from_daily_infected_model()

    try:
      new_hospital_confirmed = compiled_data.loc[date, "In hospital confirmed"]
      if np.isnan(new_hospital_confirmed):
        raise KeyError
      return new_hospital_confirmed
    except KeyError:
      return hospital_confirmed_from_daily_infected_model(date)

@brachbach model



In [1]:
# Build @brachbach model
hospital_confirmed_from_daily_infected_model = get_daily_hospital_confirmed(compiled_data, daily_infections)

@mem
def hospital_confirmed_for_date(date: date) -> int:
    """
    The total number of lab-confirmed COVID-19 patients in El Paso County in
    the hospital on this date
    """
    try:
        # Look up in-hospital confirmed cases from @KrisMoore's compiled data
        new_hospital_confirmed = compiled_data.loc[date, "In hospital confirmed"]
        if np.isnan(new_hospital_confirmed):
            raise KeyError
        return new_hospital_confirmed
    except KeyError:
        try:
            # Get point estimate from @brachbach's regression model
            cases = hospital_confirmed_from_daily_infected_model(date)

            # Add some (fairly arbitrary) uncertainty around this point estimate
            if cases == 0:
              return cases
            cases_estimate = ergo.lognormal_from_interval(cases * 0.8, cases * 1.2)
            return np.clip(cases_estimate, cases * 0.5, cases * 2)
        except KeyError:
            return 0

Proportion of ICU admissions requiring ventilation



In [1]:
@mem
def frac_icu_ventilation():
    """
    Proportion of ICU admissions requiring ventilation

    Approach (PabloStafforini et al): 
    https://pandemic.metaculus.com/questions/4154/#comment-28155

    TODO: 
    - Improve how we use case data
    - Add qualitative adjustments
    """
    ventilation_pseudocounts = 25 + 17 + 0.05 * 1150 + 0.1 * 132
    icu_pseudocounts = 100 + 36 + 0.05 * 1300 + 0.1 * 196
    return ergo.beta_from_hits(ventilation_pseudocounts, icu_pseudocounts)

## ICU admissions requiring ventilation



In [1]:
@mem
def frac_icu_ventilation():
    """
    Proportion of ICU admissions requiring ventilation

    Approach (PabloStafforini et al): 
    https://pandemic.metaculus.com/questions/4154/#comment-28155

    TODO: 
    - Improve how we use case data
    - Add qualitative adjustments
    """
    ventilation_pseudocounts = 25 + 17 + 0.05 * 1150 + 0.1 * 132
    icu_pseudocounts = 100 + 36 + 0.05 * 1300 + 0.1 * 196
    return ergo.beta_from_hits(ventilation_pseudocounts, icu_pseudocounts)

In [ ]:
@mem
def daily_infections(date: date) -> int:
    """
    What is the number of reported (new) Covid-19 infections on [date]?
    """
    try:
        # Look up Texas Government data
        new_cases = el_paso_cases.loc[date, "New cases"]
        if np.isnan(new_cases):
            raise KeyError
        return new_cases
    except KeyError:
        try:
            # Look up projections from @onlyasith's model
            cases = projected_cases.loc[date, "New cases"]
            if np.isnan(cases):
                raise KeyError
            
            # Add some (fairly arbitrary) uncertainty around this point estimate
            if cases == 0:
              return cases
            cases_estimate = ergo.lognormal_from_interval(cases * 0.8, cases * 1.2)
            return np.clip(cases_estimate, cases * 0.5, cases * 2)
        except KeyError:
            # We're beyond the time range for data and model
            return 0

@mem
def mean_infections(start_date: date, end_date: date):
    """
    What is the average number of reported new infections for this range of 
    dates? (Including start date, excluding end date)
    """
    days = daterange(start_date, end_date)
    return np.mean([daily_infections(day) for day in days])

@mem
def sma_infections(date: date):
    """
    The simple moving average of infections for a date.
    
    Defined in https://pandemic.metaculus.com/questions/4128:
    
    'The 2-day SMA is defined as the unweighted average (arithmetic mean)
    over the current day and the previous day.'
    """
    return mean_infections(date - timedelta(1), date + timedelta(1))

# Build @brachbach model
hospital_confirmed_from_daily_infected_model = get_daily_hospital_confirmed(compiled_data, daily_infections)

@mem
def hospital_confirmed_for_date(date: date) -> int:
    """
    The total number of lab-confirmed COVID-19 patients in El Paso County in
    the hospital on this date
    """
    try:
        # Look up in-hospital confirmed cases from @KrisMoore's compiled data
        new_hospital_confirmed = compiled_data.loc[date, "In hospital confirmed"]
        if np.isnan(new_hospital_confirmed):
            raise KeyError
        return new_hospital_confirmed
    except KeyError:
        try:
            # Get point estimate from @brachbach's regression model
            cases = hospital_confirmed_from_daily_infected_model(date)
            
            # Add some (fairly arbitrary) uncertainty around this point estimate
            if cases == 0:
              return cases
            cases_estimate = ergo.lognormal_from_interval(cases * 0.8, cases * 1.2)
            return np.clip(cases_estimate, cases * 0.5, cases * 2)
        except KeyError:
            return 0

@mem
def frac_icu_ventilation():
    """
    Proportion of ICU admissions requiring ventilation

    Approach (PabloStafforini et al): 
    https://pandemic.metaculus.com/questions/4154/#comment-28155

    TODO: 
    - Improve how we use case data
    - Add qualitative adjustments
    """
    ventilation_pseudocounts = 25 + 17 + 0.05 * 1150 + 0.1 * 132
    icu_pseudocounts = 100 + 36 + 0.05 * 1300 + 0.1 * 196
    return ergo.beta_from_hits(ventilation_pseudocounts, icu_pseudocounts)

@mem
def peak_compatible_with_historical_data(peak_date):
    if not peak_date in el_paso_cases.index:
        return True
    for comparison_date in daterange(START_DATE, peak_date + timedelta(11)):
        if comparison_date not in el_paso_cases.index:
            continue
        if sma_infections(comparison_date) > sma_infections(peak_date):
            return False
        if sma_infections(comparison_date) == sma_infections(peak_date) and comparison_date > peak_date:
            return False
    return True

# El Paso questions



In [1]:
@question(4137, community_weight=0.5)
def peak_infections():
    """
    How many new infections will be reported in El Paso on the day on which
    the number of new reported infections peaks?
    """
    peak = peak_infection_date()
    return daily_infections_v0(peak)
plot_question(peak_infections)

In [1]:
@question(4137, community_weight=0.5)
def peak_infections():
    """
    How many new infections will be reported in El Paso on the day on which
    the number of new reported infections peaks?
    """
    peak = peak_infection_date()
    return daily_infections_v0(peak)
plot_question(peak_infections)

In [1]:
@question(4170, community_weight=0.8)
def mean_infections_peak678():
    """
    What will the average number of reported daily infections be in El Paso,
    over the 6th, 7th and 8th days after the first "peak"?  
    """
    peak = peak_infection_date()
    return mean_infections(peak + timedelta(6), peak + timedelta(9))
plot_question(mean_infections_peak678)

In [1]:
@question(4170, community_weight=0.8)
def mean_infections_peak678():
    """
    What will the average number of reported daily infections be in El Paso,
    over the 6th, 7th and 8th days after the first "peak"?  
    """
    peak = peak_infection_date()
    return mean_infections(peak + timedelta(6), peak + timedelta(9))
plot_question(mean_infections_peak678)

In [1]:
@question(4154, community_weight=0.3)
def frac_patients_invasive():
    """
    What portion of in-hospital patients with Covid-19 in El Paso County will
    require invasive ventilation?

    Following @PabloStafforini's indirect estimation approach:
    https://pandemic.metaculus.com/questions/4154/#comment-28155

    TODO:
    - Combine with direct estimate
      direct_estimate = ergo.beta_from_hits(0.1 * 130, 0.1 * 393)
    """
    return frac_patients_icu() * frac_icu_ventilation()
plot_question(frac_patients_invasive)

In [1]:
@question(4153, community_weight=0.3)
def max_30d_hospital_confirmed_for_peak():
    """
    What will the maximum number of in-hospital lab-confirmed COVID-19 
    patients in El Paso County, in the 30-day period during which the "peak"
    occurs?
    """
    peak = peak_infection_date()
    days = daterange(peak - timedelta(15), peak + timedelta(15))
    return max(hospital_confirmed_for_date(day) for day in days)

plot_question(max_30d_hospital_confirmed_for_peak, bw=0.01)

In [1]:
@question(4204)
def peak_icu_admissions():
    """
    How many patients with Covid-19 in El Paso County will be admitted to the
    ICU on the day when the number of hospital admissions of cases peak?

    Following @Tamay's approach:
    https://pandemic.metaculus.com/questions/4204/

    Alternative:    
    - peak = peak_hospitalizations_date()
    - return daily_icu_admissions(peak)

    FIXME: Admissions vs in-hospital patients unclear

    Not mixing in community since this is just the product of two other questions.    
    """
    max_patients = max_30d_hospital_confirmed_for_peak()
    return max_patients * frac_patients_icu()
plot_question(peak_icu_admissions)

In [1]:
@question(4201)
def peak_invasive_ventilation():
    """
    How many patients with Covid-19 in El Paso County will require invasive 
    ventilation on the day when the number of hospital admissions of cases 
    peak?

    Following @Tamay's approach:
    https://pandemic.metaculus.com/questions/4201/#comment-28004

    Not mixing in community since this is just the product of two other questions.
    """
    return frac_icu_ventilation() * peak_icu_admissions()

plot_question(peak_invasive_ventilation)

# Generate predictions for all questions



In [1]:
# Generate predictions for all questions



# Compare preditions to community



# Compare predictions to community



In [1]:
This takes a while since we're fitting a mixture of logistic
distributions to our samples before visualizing (and submitting) them.
These may look a little different from the plots below the questions
above, because we've taken more samples from the distribution and we're
fitting logistic distributions so we can submit them to metaculus



In [ ]:
submissions = {}
for sampler in samplers.values():
    q = sampler.question

    q_samples = samples[sampler.__name__]

    if q.id == 4128: # Date question: Need to convert back to date from days (https://github.com/oughtinc/ergo/issues/144)
        q_samples = np.array([START_DATE + timedelta(s) for s in q_samples])

    if q.id in [4201, 4204, 4137, 4152, 4170, 4153]:
      # Clip extreme values for questions that we had issues fitting
      (sample_min, sample_max) = np.quantile(q_samples, [0.04, 0.96])
      q_samples = q_samples[(q_samples >= sample_min) & (q_samples <= sample_max)]

    submission = q.get_submission_from_samples(q_samples)
    submissions[q] = submission

    # the graph for this question will be too zoomed out unless we cut off more of the graph
    if q.id == 4153:
      q.show_prediction(q_samples, plot_samples=False, plot_fitted=True, show_community=True, percent_kept=0.7)
    else:
      q.show_prediction(q_samples, plot_samples=False, plot_fitted=True, show_community=True, percent_kept=0.9)

In [1]:
# Should we submit this to Metaculus? If so, uncomment the following lines:
# for q, submission in submissions.items():  
#     print(q.submit(submission))